## 🌍 UN SDG 7: Affordable and Clean Energy — Data Preparation

This notebook processes and integrates six datasets associated with the United Nations Sustainable Development Goal 7: **Affordable and Clean Energy**. The ultimate objective is to produce a single, clean CSV file that can be seamlessly imported into **Tableau** for impactful visualization and analysis.

---

#### 🛠️ Process Overview

##### 1. **Import Libraries**
- Load the necessary libraries — primarily `pandas` for data manipulation.

##### 2. **Load Source Data**
- Read in six individual CSV files containing energy-related metrics.

##### 3. **Clean and Standardize**
- For each dataset:
  - Rename the primary metric column for clarity and consistency.
  - Fill in missing `Code` values for entities not classified as countries.

##### 4. **Merge Datasets**
- Use an **outer join** to combine all six datasets into a single master DataFrame.
- This ensures no data point is excluded during the merge.

##### 5. **Export Final Dataset**
- Save the cleaned and merged DataFrame to a new `.csv` file for future use in Tableau or other visualization tools.

---

### ✅ Outcome
A well-structured, unified CSV file that supports deeper insights and data storytelling around global energy access and sustainability.


In [1]:
import pandas as pd
from functools import reduce
import numpy as np
import warnings
import os
from pathlib import Path

# Suppress warnings for a cleaner output
warnings.filterwarnings('ignore')

print("Libraries imported successfully.")

Libraries imported successfully.


### 2. Load and Pre-process Datasets
---

- First, we define a dictionary to hold the file paths and the desired new, shorter column name for the main metric in each file. This makes the process scalable and easy to read.

In [2]:
# Dictionary mapping file paths to their specific data column and a new, clean name

# --- CHANGE HERE ---
# Define the path to your raw data folder
# The '..' means "go up one directory" from /notebooks/ to the project root
# Then go into the /data/raw/ folder.
data_path = 'D:/github/python projects/Analysis/sustainable_goal_affordable_ and_clean_renewable_enargy/Analysis/Affordable_Clean_Energy_Project/data/raw' 

# Dictionary mapping file names to their column info
datasets_info = {
    'access-to-clean-fuels-and-technologies-for-cooking.csv': {
        'original_col': 'Proportion of population with primary reliance on clean fuels and technologies for cooking (%) - Residence area type: Total',
        'new_col': 'Access_Clean_Fuels_Pct'
    },
    'energy-intensity-of-economies.csv': {
        'original_col': 'Energy intensity level of primary energy (MJ/$2017 PPP GDP)',
        'new_col': 'Energy_Intensity_MJ_per_GDP'
    },
    'international-finance-clean-energy.csv': {
        'original_col': '7.a.1 - International financial flows to developing countries in support of clean energy research and development and renewable energy production, including in hybrid systems (millions of constant 2021 United States dollars) - EG_IFF_RANDN - All renewables',
        'new_col': 'Clean_Energy_Finance_Flows_USD'
    },
    'renewable-electricity-generating-capacity-per-capita.csv': {
        'original_col': '7.b.1 - Installed renewable electricity-generating capacity (watts per capita) - EG_EGY_RNEW - All renewables',
        'new_col': 'Renewable_Capacity_per_Capita_Watts'
    },
    'share-of-final-energy-consumption-from-renewable-sources.csv': {
        'original_col': '7.2.1 - Renewable energy share in the total final energy consumption (%) - EG_FEC_RNEW',
        'new_col': 'Renewable_Energy_Share_Pct'
    },
    'share-of-the-population-with-access-to-electricity.csv': {
        'original_col': 'Access to electricity (% of population)',
        'new_col': 'Access_to_Electricity_Pct'
    }
}

# List to store the processed dataframes
data_frames = []



#### The cleaning process involves:

- Renaming the long, descriptive column to a short, computer-friendly name.

- Fixing Missing Codes: For rows where the Code is empty (like for "Africa (UN)"), we'll generate a custom code from the Entity name (e.g., "AFRICA_UN"). This ensures we can still use these rows effectively.

In [3]:
for file, info in datasets_info.items():
    try:
        # --- CHANGE HERE ---
        # Construct the full path to the file
        full_path = os.path.join(data_path, file)
        
        # Load the dataset using the full path
        df = pd.read_csv(full_path)
        
        # --- 1. Standardize Column Names ---
        df.rename(columns={info['original_col']: info['new_col']}, inplace=True)
        
        # Ensure the base columns ('Entity', 'Code', 'Year') exist
        base_cols = ['Entity', 'Code', 'Year']
        df = df[base_cols + [info['new_col']]]

        # --- 2. Fix Missing 'Code' Values ---
        missing_code_mask = df['Code'].isnull() & df['Entity'].notnull()
        df.loc[missing_code_mask, 'Code'] = df.loc[missing_code_mask, 'Entity'].apply(
            lambda x: x.upper().replace(' ', '_').replace('(', '').replace(')', '')
        )
        
        # --- 3. Append to our list of dataframes ---
        data_frames.append(df)
        
        print(f"Successfully processed: {file}")
        print(f"  - Renamed column to '{info['new_col']}'")
        print(f"  - Shape: {df.shape}\n")
        
    except FileNotFoundError:
        print(f"Error: The file '{full_path}' was not found. Please check your file path and name.")
    except KeyError:
        print(f"Error: Column '{info['original_col']}' not found in '{file}'. Please check column names.")


Successfully processed: access-to-clean-fuels-and-technologies-for-cooking.csv
  - Renamed column to 'Access_Clean_Fuels_Pct'
  - Shape: (5148, 4)

Successfully processed: energy-intensity-of-economies.csv
  - Renamed column to 'Energy_Intensity_MJ_per_GDP'
  - Shape: (4799, 4)

Successfully processed: international-finance-clean-energy.csv
  - Renamed column to 'Clean_Energy_Finance_Flows_USD'
  - Shape: (3772, 4)

Successfully processed: renewable-electricity-generating-capacity-per-capita.csv
  - Renamed column to 'Renewable_Capacity_per_Capita_Watts'
  - Shape: (5474, 4)

Successfully processed: share-of-final-energy-consumption-from-renewable-sources.csv
  - Renamed column to 'Renewable_Energy_Share_Pct'
  - Shape: (5290, 4)

Successfully processed: share-of-the-population-with-access-to-electricity.csv
  - Renamed column to 'Access_to_Electricity_Pct'
  - Shape: (6707, 4)



In [4]:
# Check if we have dataframes to merge
if not data_frames:
    print("No dataframes were loaded. Halting execution.")
else:
    # Use functools.reduce for a clean and efficient merge of all dataframes in the list
    merged_df = reduce(lambda left, right: pd.merge(
        left, right, on=['Entity', 'Code', 'Year'], how='outer'
    ), data_frames)

    # --- Post-Merge Cleanup ---
    # Sort the data for better readability and structure
    merged_df.sort_values(by=['Entity', 'Year'], inplace=True)

    # Ensure 'Year' is an integer data type
    # Using 'Int64' (capital I) to allow for potential NaNs if a row somehow misses a year
    merged_df['Year'] = merged_df['Year'].astype('Int64')

    print("All datasets have been merged successfully!")
    print("\n--- Merged DataFrame Info ---")
    merged_df.info()

All datasets have been merged successfully!

--- Merged DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8198 entries, 0 to 8197
Data columns (total 9 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Entity                               8198 non-null   object 
 1   Code                                 8198 non-null   object 
 2   Year                                 8198 non-null   Int64  
 3   Access_Clean_Fuels_Pct               5148 non-null   float64
 4   Energy_Intensity_MJ_per_GDP          4799 non-null   float64
 5   Clean_Energy_Finance_Flows_USD       3772 non-null   float64
 6   Renewable_Capacity_per_Capita_Watts  5474 non-null   float64
 7   Renewable_Energy_Share_Pct           5290 non-null   float64
 8   Access_to_Electricity_Pct            6707 non-null   float64
dtypes: Int64(1), float64(6), object(2)
memory usage: 584.6+ KB


In [5]:
# --- DEFINE OUTPUT PATH ---
output_dir = Path.cwd().parent / "data" / "processed"

# Create the directory if it doesn't exist yet (good practice)
output_dir.mkdir(parents=True, exist_ok=True)

# Define the full path for the output file
output_filepath = output_dir / 'affordable_and_clean_energy_merged.csv'


# --- SAVE THE DATAFRAME ---
merged_df.to_csv(output_filepath, index=False)


# --- DISPLAY RESULTS ---
print(f"✅ Final dataset saved successfully at:\n{output_filepath}")

print("\nHere's a preview of the first 10 rows of your merged data:")
display(merged_df.head(10))

print("\nAnd a preview of the last 10 rows:")
display(merged_df.tail(10))

✅ Final dataset saved successfully at:
d:\github\python projects\Analysis\sustainable_goal_affordable_ and_clean_renewable_enargy\Analysis\Affordable_Clean_Energy_Project\data\processed\affordable_and_clean_energy_merged.csv

Here's a preview of the first 10 rows of your merged data:


,Entity,Code,Year,Access_Clean_Fuels_Pct,Energy_Intensity_MJ_per_GDP,Clean_Energy_Finance_Flows_USD,Renewable_Capacity_per_Capita_Watts,Renewable_Energy_Share_Pct,Access_to_Electricity_Pct
0,Afghanistan,AFG,2000,6.20,1.50,30000.0,9.80,44.99,4.4
1,Afghanistan,AFG,2001,7.10,1.50,140000.0,9.73,45.60,9.3
2,Afghanistan,AFG,2002,8.30,1.40,4390000.0,9.12,37.83,14.1
3,Afghanistan,AFG,2003,9.35,1.39,28330000.0,8.46,36.66,19.0
4,Afghanistan,AFG,2004,10.80,1.21,0.0,8.14,44.24,23.8
5,Afghanistan,AFG,2005,12.10,1.41,10770000.0,7.89,33.88,28.7
6,Afghanistan,AFG,2006,13.60,1.50,11620000.0,7.69,31.89,33.5
7,Afghanistan,AFG,2007,15.30,1.54,17530000.0,7.58,28.78,38.4
8,Afghanistan,AFG,2008,16.70,1.95,18000000.0,7.86,21.17,42.4
9,Afghanistan,AFG,2009,18.30,2.27,11020000.0,7.77,16.53,48.3



And a preview of the last 10 rows:


,Entity,Code,Year,Access_Clean_Fuels_Pct,Energy_Intensity_MJ_per_GDP,Clean_Energy_Finance_Flows_USD,Renewable_Capacity_per_Capita_Watts,Renewable_Energy_Share_Pct,Access_to_Electricity_Pct
8188,Zimbabwe,ZWE,2014,29.8,14.69,19650000.0,62.07,80.75,32.3
8189,Zimbabwe,ZWE,2015,29.9,14.59,0.0,61.81,80.78,33.7
8190,Zimbabwe,ZWE,2016,29.9,14.18,40000.0,60.82,81.68,42.5
8191,Zimbabwe,ZWE,2017,29.9,13.62,6450000.0,60.00,81.98,44.0
8192,Zimbabwe,ZWE,2018,30.1,13.64,10000.0,78.97,79.65,45.4
8193,Zimbabwe,ZWE,2019,30.1,14.35,270000.0,77.62,81.04,46.7
8194,Zimbabwe,ZWE,2020,30.3,15.05,60000.0,76.20,84.06,52.7
8195,Zimbabwe,ZWE,2021,30.3,14.77,10000.0,75.53,82.38,49.0
8196,Zimbabwe,ZWE,2022,30.5,NaN,3750000.0,74.83,NaN,50.1
8197,Zimbabwe,ZWE,2023,30.7,NaN,NaN,NaN,NaN,NaN
